Nous allons utiliser la bibliothèque [Beautiful Soup (`bs4`)](https://beautiful-soup-4.readthedocs.io/en/latest/) pour `python`

In [ ]:
import bs4 #Beautiful Soup 4
import requests #library to send HTTP requests

## Introduction

Nous allons essayer d'extraire des informations sur des films référencés sur [IMDB](https://www.imdb.com/search/title?release_date=2018&sort=boxoffice_gross_us,desc&start=1)

In [ ]:
url = 'https://www.imdb.com/search/title?release_date=2018&sort=boxoffice_gross_us,desc&start=1'

source = requests.get(url).text
soup = bs4.BeautifulSoup(source,'html.parser')

Que contient la variable `soup` ?

In [ ]:
#


Avec la méthode `find_all`, on peut filtrer le document `soup`.

En s'appuyant sur la [documentation](https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=find_all#find-all) de la méthode `find_all` de Beautiful Soup, on peut filtrer le document avec les filtres `name = "div"` et `attrs={"class";"list-item-content"}`

In [ ]:
#movie_blocks =

En affichant, on peut remarquer qu'entre la première balise `<a>...</a>`, apparaît le titre du film.

Extraire le titre du film avec la méthode `find` sur le premier bloc `movie_blocks[0]` et en extraire le texte qu'il contient avec la méthode `get_text`.

In [ ]:
# mname = 

De la même manière, on peut extraire l'année de sortie. L'année de sortie apparaît dans la balise `span` avec pour attribut `class="lister-item-year"`. On récupère la liste du contenu de la balise avec `.contents`


L'année ainsi récupérée est entre parenthèses, on veillera donc à ne pas conserver le premier et le dernier caractère de la chaîne de caractères extraite (avec par exemple la tranche d'indices `[1:-1]`)

In [ ]:
# m_reyear =

## Exemples

In [ ]:
import time
import random
import pandas as pd

In [ ]:
def scrape_mblock(movie_block):
    
    movieb_data ={}
  
    try:
        movieb_data['name'] = movie_block.find('a').get_text() # Name of the movie
    except:
        movieb_data['name'] = None

    try:    
        movieb_data['year'] = str(movie_block.find('span',{'class': 'lister-item-year'}).contents[0][1:-1]) # Release year
    except:
        movieb_data['year'] = None

    try:
        movieb_data['rating'] = float(movie_block.find('div',{'class':'inline-block ratings-imdb-rating'}).get('data-value')) #rating
    except:
        movieb_data['rating'] = None

    try:
        movieb_data['m_score'] = float(movie_block.find('span',{'class':'metascore favorable'}).contents[0].strip()) #meta score
    except:
        movieb_data['m_score'] = None

    try:
        movieb_data['votes'] = int(movie_block.find('span',{'name':'nv'}).get('data-value')) # votes
    except:
        movieb_data['votes'] = None

    return movieb_data

In [ ]:
def scrape_m_page(movie_blocks):
    
    page_movie_data = []
    num_blocks = len(movie_blocks)
    
    for block in range(num_blocks):
        page_movie_data.append(scrape_mblock(movie_blocks[block]))
    
    return page_movie_data

In [ ]:
def scrape_this(link,t_count):

    base_url = link
    target = t_count
    
    current_mcount_start = 0
    current_mcount_end = 0
    remaining_mcount = target - current_mcount_end 
    
    new_page_number = 1
    
    movie_data = []
    
    
    while remaining_mcount > 0:

        url = base_url + str(new_page_number)
        
        #set_trace()
        
        source = requests.get(url).text
        soup = bs4.BeautifulSoup(source,'html.parser')
        
        movie_blocks = soup.findAll('div',{'class':'lister-item-content'})
        
        movie_data.extend(scrape_m_page(movie_blocks))   
        
        current_mcount_start = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[0])

        current_mcount_end = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[1].split(" ")[0])

        remaining_mcount = target - current_mcount_end
        
        print('\r' + "currently scraping movies from: " + str(current_mcount_start) + " - "+str(current_mcount_end), "| remaining count: " + str(remaining_mcount), flush=True, end ="")
        
        new_page_number = current_mcount_end + 1
        
        time.sleep(random.randint(0, 10))
    
    return movie_data

In [ ]:
base_scraping_link = "https://www.imdb.com/search/title?release_date=2018-01-01,2018-12-31&sort=boxoffice_gross_us,desc&start="

top_movies = 150 #input("How many movies do you want to scrape?")
films = []

films = scrape_this(base_scraping_link,int(top_movies))

print('\r'+"List of top " + str(top_movies) +" movies:" + "\n", end="\n")
pd.DataFrame(films)

List of top 150 movies:



,name,year,rating,m_score,votes
0,Black Panther,2018,7.3,88.0,713990
1,Avengers: Infinity War,2018,8.5,68.0,997797
2,Incredibles 2,2018,7.6,80.0,282417
3,Jurassic World: Fallen Kingdom,2018,6.2,NaN,294856
4,Aquaman,2018,6.9,NaN,437260
...,...,...,...,...,...
145,Boy Erased,2018,6.9,69.0,37677
146,Hotel Artemis,2018,6.1,NaN,52265
147,A-X-L,2018,5.3,NaN,11702
148,Run the Race,2018,6.0,NaN,1496


## Travail en autonomie

Maintenant à vous de jouer ! Utilisez la bibliothèque Beautiful Soup et allez extraire des informations textuelles provenant d'un site.

Quelques exemples de pages :

* Wikipédia

* [GoodReads best books ever](https://www.goodreads.com/list/show/1.Best_Books_Ever)

* [All Side media bias rating](https://www.allsides.com/media-bias/media-bias-ratings)

* [NASA Mars news website](https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest)

* [Mars weather Twitter account](https://twitter.com/marswxreport?lang=en) /!\ visiblement il est plus facile d'extraire des données de Twitter quand on y est pas connecté !

* [Mars facts](https://space-facts.com/mars/)

* [Fake Python Jobs](https://realpython.github.io/fake-jobs/)


Comme dans l'exemple précédent, vous pouvez ensuite structurer ces données dans un tableau et en faire un *dataframe* avec la bibliothèque `pandas`